In [2]:
import numpy as np
import keras
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 5.2 MB/s 


In [4]:
import keras_tuner as kt

In [5]:
from google.colab import files
upload = files.upload()

Saving heart.csv to heart.csv


#Cek Data

In [6]:
df = pd.read_csv("heart.csv")

df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [7]:
#cek kolom

df.columns

Index(['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall', 'output'],
      dtype='object')

In [8]:
#cek korelasi data awal

print(df.corr())

               age       sex        cp    trtbps      chol       fbs  \
age       1.000000 -0.098447 -0.068653  0.279351  0.213678  0.121308   
sex      -0.098447  1.000000 -0.049353 -0.056769 -0.197912  0.045032   
cp       -0.068653 -0.049353  1.000000  0.047608 -0.076904  0.094444   
trtbps    0.279351 -0.056769  0.047608  1.000000  0.123174  0.177531   
chol      0.213678 -0.197912 -0.076904  0.123174  1.000000  0.013294   
fbs       0.121308  0.045032  0.094444  0.177531  0.013294  1.000000   
restecg  -0.116211 -0.058196  0.044421 -0.114103 -0.151040 -0.084189   
thalachh -0.398522 -0.044020  0.295762 -0.046698 -0.009940 -0.008567   
exng      0.096801  0.141664 -0.394280  0.067616  0.067023  0.025665   
oldpeak   0.210013  0.096093 -0.149230  0.193216  0.053952  0.005747   
slp      -0.168814 -0.030711  0.119717 -0.121475 -0.004038 -0.059894   
caa       0.276326  0.118261 -0.181053  0.101389  0.070511  0.137979   
thall     0.068001  0.210041 -0.161736  0.062210  0.098803 -0.03

In [9]:
#cek missing values
df.isnull().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

In [10]:
#cek value yang berbeda dari masing-masing atribut
df["thall"].unique()

array([1, 2, 3, 0])

In [11]:
#cek describe data

df.describe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [12]:
#cek outlier pada masing-masing atribut

quantile1, quantile3 = np.percentile(df.trtbps,[25,75])
iqr_value = quantile3 - quantile1
lower = quantile1 - (1.5*iqr_value)
upper = quantile3 + (1.5*iqr_value)

print(lower, upper)

90.0 170.0


#Data Cleaning

In [13]:
#Ambil Interval dari batas bawah dan batas atas
df1 = df.loc[(df['trtbps'] <= 170)&(df['chol'] <= 369.75)&(df['thalachh'] >= 84.75)&(df['oldpeak'] <= 4)&(df['caa']<=2.5)&(df['thall'] >= 0.5)]

df1.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Batas dari tiap atribut yang diambil: 

$ \\ trtbps <= 170 \\ chol <= 369.75 \\ thalachh >= 84.75 \\ oldpeak <= 4 \\ caa <= 2.5 \\ thall >= 0.5$




In [14]:
#cek describe df1

df1.describe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000
mean,53.777778,0.685824,1.000000,129.478927,242.022989,0.126437,0.532567,150.992337,0.314176,0.955556,1.432950,0.509579,2.298851,0.578544
std,9.044571,0.465079,1.037749,15.448567,44.402957,0.332979,0.515055,22.562220,0.465079,1.038609,0.607886,0.721180,0.577427,0.494741
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,88.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,46.000000,0.000000,0.000000,120.000000,209.000000,0.000000,0.000000,138.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,239.000000,0.000000,1.000000,155.000000,0.000000,0.600000,1.000000,0.000000,2.000000,1.000000
75%,60.000000,1.000000,2.000000,140.000000,269.000000,0.000000,1.000000,168.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,76.000000,1.000000,3.000000,170.000000,360.000000,1.000000,2.000000,202.000000,1.000000,4.000000,2.000000,2.000000,3.000000,1.000000


In [15]:
#cek outlier pada masing-masing atribut df1

quantile1, quantile3 = np.percentile(df1.thall,[25,75])
iqr_value = quantile3 - quantile1
lower = quantile1 - (1.5*iqr_value)
upper = quantile3 + (1.5*iqr_value)

print(lower, upper)

0.5 4.5


Batas dari tiap atribut yang diambil:

$chol <= 359 \\ thalachh >= 93$

In [16]:
#Ambil Interval dari batas bawah dan batas atas
df2 = df1.loc[(df1['chol'] <= 359)&(df1['thalachh'] >= 93)]

df2.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [17]:
#cek describe df2

df2.describe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000
mean,53.701550,0.686047,1.003876,129.139535,241.697674,0.124031,0.534884,151.472868,0.313953,0.955039,1.434109,0.503876,2.302326,0.581395
std,9.061832,0.465000,1.038174,15.201753,43.811340,0.330258,0.515088,22.019297,0.465000,1.044494,0.609203,0.718017,0.573129,0.494289
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,95.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,46.000000,0.000000,0.000000,120.000000,209.250000,0.000000,0.000000,138.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,54.000000,1.000000,1.000000,130.000000,239.000000,0.000000,1.000000,155.000000,0.000000,0.600000,1.000000,0.000000,2.000000,1.000000
75%,60.000000,1.000000,2.000000,140.000000,269.000000,0.000000,1.000000,168.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,76.000000,1.000000,3.000000,170.000000,354.000000,1.000000,2.000000,202.000000,1.000000,4.000000,2.000000,2.000000,3.000000,1.000000


In [18]:
#cek outlier pada masing-masing atribut df2

quantile1, quantile3 = np.percentile(df2.thalachh,[25,75])
iqr_value = quantile3 - quantile1
lower = quantile1 - (1.5*iqr_value)
upper = quantile3 + (1.5*iqr_value)

print(lower, upper)

93.0 213.0


df2 sudah bersih dari outlier dan tidak ada missing values. maka, dapat digunakan df2 untuk kita olah

#Preprocessing Data

In [19]:
#cek korelasi setelah pembersihan Data

print(df2.corr())

               age       sex        cp    trtbps      chol       fbs  \
age       1.000000 -0.069417 -0.036687  0.269206  0.126272  0.128132   
sex      -0.069417  1.000000 -0.086131  0.006221 -0.122332  0.051853   
cp       -0.036687 -0.086131  1.000000  0.116830 -0.050276  0.157473   
trtbps    0.269206  0.006221  0.116830  1.000000  0.087588  0.124420   
chol      0.126272 -0.122332 -0.050276  0.087588  1.000000 -0.000894   
fbs       0.128132  0.051853  0.157473  0.124420 -0.000894  1.000000   
restecg  -0.088209 -0.092184  0.068872 -0.122371 -0.120918 -0.071280   
thalachh -0.406364 -0.044728  0.264261  0.000209 -0.003465  0.014911   
exng      0.091579  0.169701 -0.373298 -0.013377  0.050517 -0.001178   
oldpeak   0.197959  0.168705 -0.120406  0.157895 -0.024949  0.024125   
slp      -0.145601 -0.052707  0.071156 -0.058245  0.045319 -0.075260   
caa       0.387995  0.079411 -0.200986  0.041302  0.107527  0.096418   
thall     0.072132  0.255335 -0.191622  0.009877  0.070133 -0.05

In [20]:
#cek describe data
df2.describe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000
mean,53.701550,0.686047,1.003876,129.139535,241.697674,0.124031,0.534884,151.472868,0.313953,0.955039,1.434109,0.503876,2.302326,0.581395
std,9.061832,0.465000,1.038174,15.201753,43.811340,0.330258,0.515088,22.019297,0.465000,1.044494,0.609203,0.718017,0.573129,0.494289
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,95.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,46.000000,0.000000,0.000000,120.000000,209.250000,0.000000,0.000000,138.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,54.000000,1.000000,1.000000,130.000000,239.000000,0.000000,1.000000,155.000000,0.000000,0.600000,1.000000,0.000000,2.000000,1.000000
75%,60.000000,1.000000,2.000000,140.000000,269.000000,0.000000,1.000000,168.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,76.000000,1.000000,3.000000,170.000000,354.000000,1.000000,2.000000,202.000000,1.000000,4.000000,2.000000,2.000000,3.000000,1.000000


Ambil fitur yang korelasinya yang positif terhadap output. yaitu, cp, fbs, restecg, thalachh, slp

In [21]:
#bagi fitur yang dipilih dengan oleh nilai maksimumnya (proses skaling)

df2["cp"], df2["restecg"], df2["thalachh"], df2["slp"] = df2["cp"]/3 ,  df2["restecg"]/2 , df2["thalachh"]/202 , df2["slp"]/2
df2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,1.000000,145,233,1,0.0,0.742574,0,2.3,0.0,0,1,1
1,37,1,0.666667,130,250,0,0.5,0.925743,0,3.5,0.0,0,2,1
2,41,0,0.333333,130,204,0,0.0,0.851485,0,1.4,1.0,0,2,1
3,56,1,0.333333,120,236,0,0.5,0.881188,0,0.8,1.0,0,2,1
4,57,0,0.000000,120,354,0,0.5,0.806931,1,0.6,1.0,0,2,1


In [22]:
#definisikan fitur-fitur yang akan kita olah
X = df2[['cp', 'fbs', 'restecg', 'thalachh', 'slp']].values
X[0:5]

array([[1.        , 1.        , 0.        , 0.74257426, 0.        ],
       [0.66666667, 0.        , 0.5       , 0.92574257, 0.        ],
       [0.33333333, 0.        , 0.        , 0.85148515, 1.        ],
       [0.33333333, 0.        , 0.5       , 0.88118812, 1.        ],
       [0.        , 0.        , 0.5       , 0.80693069, 1.        ]])

In [23]:
#definisikan kelasnya
y = df2[['output']].values

y[100:200]

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

#Train/Tes Split

In [24]:
#pemilihan/pemisahan data latih dan data tes
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)

In [25]:
#tampilkan ukuran data latih dan testnya
print("X_train_full: ", X_train_full.shape)
print("y_train_full: ", y_train_full.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

X_train_full:  (206, 5)
y_train_full:  (206, 1)
X_test:  (52, 5)
y_test:  (52, 1)


In [26]:
#definisikan validasi data dan data latih baru bagian data latih sebelumnya
X_valid, X_train = X_train_full[:78], X_train_full[78:]
y_valid, y_train = y_train_full[:78], y_train_full[78:]

In [27]:
#cek kelas apa saja nantinya
df["output"].unique()

array([1, 0])

In [28]:
#definisikan kelas
class_heart = [0,1]

#Modelling

In [29]:
#membangun model klasifikasi dengan hyperparameter
def build_model(hp):
  model = keras.models.Sequential()
  hp_layers = hp.Choice('layer', [2,3]) #banyak hidden layer 2 atau 3
  hp_neurons = hp.Int('neuron', min_value = 100, max_value=300, step = 100) #banyak neuron disetiap layer 100, 20

  for layer in range(hp_layers):
    model.add(keras.layers.Dense(hp_neurons, activation = 'relu')) #hidden layer
  model.add(keras.layers.Dense(10, activation = 'softmax')) #output layer 

  optimizer = tf.keras.optimizers.Adam()
  model.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

  return model

In [30]:
tuner = kt.Hyperband(build_model, objective = 'val_accuracy', max_epochs = 10, directory = 'my_dir', project_name = 'my_tuner')

In [31]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 5)

In [32]:
tuner.search(X_train, y_train, epochs = 50, validation_data = [X_valid, y_valid], callbacks = [stop_early])

Trial 6 Complete [00h 00m 01s]
val_accuracy: 0.6410256624221802

Best val_accuracy So Far: 0.7564102411270142
Total elapsed time: 00h 00m 07s
INFO:tensorflow:Oracle triggered exit


In [33]:
best_hps = tuner.get_best_hyperparameters()[0]
print("best number of layers: ", best_hps.get('layer'))
print("best number of neurons: ", best_hps.get('neuron'))

best number of layers:  3
best number of neurons:  300


In [34]:
model = tuner.hypermodel.build(best_hps)

In [35]:
model.fit(X_train, y_train, epochs = 150, validation_data = [X_valid, y_valid])

Epoch 1/150
4/4 [==============================] - 1s 80ms/step - loss: 2.1580 - accuracy: 0.3984 - val_loss: 1.8394 - val_accuracy: 0.6410
Epoch 2/150
4/4 [==============================] - 0s 16ms/step - loss: 1.6463 - accuracy: 0.5469 - val_loss: 1.2102 - val_accuracy: 0.6410
Epoch 3/150
4/4 [==============================] - 0s 15ms/step - loss: 1.0606 - accuracy: 0.5469 - val_loss: 0.7308 - val_accuracy: 0.6410
Epoch 4/150
4/4 [==============================] - 0s 16ms/step - loss: 0.7419 - accuracy: 0.5469 - val_loss: 0.6194 - val_accuracy: 0.6795
Epoch 5/150
4/4 [==============================] - 0s 20ms/step - loss: 0.6668 - accuracy: 0.5781 - val_loss: 0.6153 - val_accuracy: 0.7821
Epoch 6/150
4/4 [==============================] - 0s 16ms/step - loss: 0.6087 - accuracy: 0.6328 - val_loss: 0.5406 - val_accuracy: 0.6667
Epoch 7/150
4/4 [==============================] - 0s 19ms/step - loss: 0.6162 - accuracy: 0.6406 - val_loss: 0.5559 - val_accuracy: 0.7821
Epoch 8/150
4/4 [===

#Evaluasi Model

In [36]:
#cek akurasi dan error
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 9ms/step - loss: 1.6118 - accuracy: 0.6538


[1.6117615699768066, 0.6538461446762085]

In [37]:
#hitung peluang apakah data masuk ke kelas 0 atau 1
y_proba = model.predict(X_test)
y_proba

array([[1.68599170e-02, 9.83097851e-01, 8.08531058e-06, 4.26112274e-06,
        5.06029028e-06, 3.42243243e-06, 5.44715431e-06, 7.54236407e-06,
        4.33364039e-06, 4.05906849e-06],
       [9.39340472e-01, 6.05494678e-02, 1.47372730e-05, 1.54634836e-05,
        1.52211887e-05, 1.21388857e-05, 1.64395988e-05, 1.14486274e-05,
        1.05256267e-05, 1.41387845e-05],
       [7.51159072e-01, 2.47748896e-01, 1.53240078e-04, 1.49534753e-04,
        1.47298866e-04, 1.24295984e-04, 1.46548962e-04, 1.24927727e-04,
        1.13609334e-04, 1.32583329e-04],
       [7.00776100e-01, 2.98517704e-01, 9.98016476e-05, 9.52753689e-05,
        9.56109579e-05, 7.92014034e-05, 9.75608928e-05, 8.29131895e-05,
        7.33399138e-05, 8.24604795e-05],
       [7.68546537e-02, 9.23124433e-01, 4.08500819e-06, 2.35922130e-06,
        2.47154708e-06, 1.92183097e-06, 2.60970955e-06, 2.93736707e-06,
        2.25599024e-06, 2.32082721e-06],
       [1.31131435e-06, 9.99998689e-01, 1.14159945e-10, 1.48426705e-11,
   

In [38]:
#Hasil Prediksi
y_p = np.array([np.argmax(i) for i in y_proba]) #mendapatkan kelas
print(y_p)

[1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1
 0 1 0 1 1 0 1 0 1 1 0 0 1 0 1]


In [39]:
print(np.array(class_heart)[y_p])

[1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1
 0 1 0 1 1 0 1 0 1 1 0 0 1 0 1]


In [40]:
#cek precision, recall, dan f-1 score
print(classification_report(y_test, y_p))

              precision    recall  f1-score   support

           0       0.59      0.59      0.59        22
           1       0.70      0.70      0.70        30

    accuracy                           0.65        52
   macro avg       0.65      0.65      0.65        52
weighted avg       0.65      0.65      0.65        52



Algoritma:

#Pengecekan Data
1. Import module yang dibutuhkan
2. upload file dataset
3. Lakukan pengecekan data, apakah ada missing value dan outlier atau tidak. Cek juga korelasi sebelum dilakukan pembersihan data
#Data Cleaning
4. Deteksi outlier dengan menggunakan metode interquartile, lalu hapus.
#Preprocessing Data
5. Cek korelasi setelah pembersihan data
6. Ambil fitur yang korelasinya positif terhadap output. Artinya, pilih fitur yang memiliki hubungan dengan atribut output
7. Bagi fitur yang terpilih oleh nilai maksimumnya. Tujuannya, agar datanya tidak memiliki rentang yang jauh sehingga dapat dilakukan proses klasifikasi dengan baik
8. Definisikan X sebagai kumpulan fitur yang terpilih dan y sebagai kelasnya
#Splitting Data
9. Definisikan pemilihan data training, test, dan validasi data
10. definsikan kelasnya, yaitu 0 dan 1.
#Modelling
11. Memodelkan klasifikasi dan menentukan hyperparameter terbaik supaya didapatkan model klasifikasi terbaik
#Evaluasi model
12. Mencari error, akurasi, f-1 score, precission, dan recall
#Analisis accuracy, precision, recall dan f1-score
didapatkan accuracy sebesar 0.65. Artinya, pemodelan klasifikasi ini cukup baik karena nilai akurasinya masih di atas 0.6 . precision, recall dan f1-score untuk kelas 0 semuanya bernilai 0.59 . serta precision, recall dan f1-score untuk kelas 1 semuanya bernilai 0.70. Perhatikan bahwa dari nilai ketiganya, hanya kelas 0 lah yang kurang dari 0.6. Bisa jadi, hal ini mungkin terjadi karena lebih banyaknya pada data tes yang kelasnya bernilai 1.
